In [ ]:
import spatialdata
import sopa.segmentation
import sopa.io
from multiprocessing import Pool

Load in the data

In [ ]:
path = "/home/fotakis/myScratch/MACSima/data/test_run/new_test/"
sdata = sopa.io.macsima(path)

In [ ]:
sdata

In [ ]:
sdata = sopa.io.macsima(path)

In [ ]:
image_key = "new_test"
# points_key = "transcripts" # (ignore this for multiplex imaging)
# gene_column = "genes" # (optional) column of sdata[points_key] containing the gene names

In [ ]:
patches = sopa.segmentation.Patches2D(sdata, image_key, patch_width=3000, patch_overlap=40)
patches.write();

In [ ]:
from sopa._sdata import get_spatial_image

print(get_spatial_image(sdata, image_key).c.values)

In [ ]:
channels = ["DAPI"]

method = sopa.segmentation.methods.cellpose_patch(diameter=35, channels=channels, flow_threshold=2, cellprob_threshold=-6)

segmentation = sopa.segmentation.StainingSegmentation(sdata, method, channels, min_area=400)

# The cellpose boundaries will be temporary saved here. You can choose a different path
cellpose_temp_dir = "test_run/.sopa_cache/cellpose"

In [ ]:
segmentation.write_patches_cells(cellpose_temp_dir)

In [ ]:
# Parallel
# Function to process a single Loom file (will be used for multiprocessing pool)
def cellpose_segmentation(patch_index):
    print("processing patch: " + str(patch_index) + "\n")
    segmentation.write_patch_cells(cellpose_temp_dir, patch_index)
    # return ladata

patches_list = list(range(len(sdata['sopa_patches'])))
# Number of processes to use
# num_processes = os.cpu_count()  # Run only if you want to use the number of all available CPU cores
num_processes = 6

# Create a multiprocessing Pool
with Pool(processes=num_processes) as pool:
    pool.map(cellpose_segmentation, patches_list)

Resolving conflicts

In [ ]:
cells = sopa.segmentation.StainingSegmentation.read_patches_cells(cellpose_temp_dir)
cells = sopa.segmentation.shapes.solve_conflicts(cells)

shapes_key = "cellpose_boundaries" # name of the key given to the cells in sdata.shapes

sopa.segmentation.StainingSegmentation.add_shapes(sdata, cells, image_key, shapes_key)